In [ ]:
import duckdb
import pandas as pd

In [2]:
df_mediation_num = duckdb.read_csv("./data/raw/mediation_numerique.csv")
df_mediation_num

┌────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────┬───────────────────────────────────────────────────────────────────────┬────────────────────┬─────────────┬────────────┬───────────────────────────────┬────────────────────┬───────────────┬──────────────┬──────────────┬───────────────┬───────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────

In [4]:
query = """ 
SELECT 
    count(id) AS nb_mediation,
    code_insee
FROM df_mediation_num
GROUP BY code_insee
"""

df_mediation_num_grouped = duckdb.sql(query)
df_mediation_num_grouped

┌──────────────┬────────────┐
│ nb_mediation │ code_insee │
│    int64     │  varchar   │
├──────────────┼────────────┤
│            4 │ 41059      │
│           35 │ 34172      │
│          508 │ 75056      │
│            1 │ 71047      │
│            1 │ 51308      │
│            3 │ 02555      │
│            1 │ 13076      │
│            4 │ 01269      │
│            2 │ 51510      │
│            1 │ 86290      │
│            · │   ·        │
│            · │   ·        │
│            · │   ·        │
│            1 │ 36035      │
│            1 │ 41050      │
│            1 │ 50394      │
│            1 │ 66084      │
│            1 │ 66186      │
│            1 │ 70154      │
│            1 │ 68298      │
│            1 │ 64132      │
│            1 │ 73151      │
│            1 │ 82046      │
├──────────────┴────────────┤
│   6957 rows (20 shown)    │
└───────────────────────────┘

In [28]:
duckdb.sql("""select sum(nb_mediation) from df_mediation_num_grouped""")

┌───────────────────┐
│ sum(nb_mediation) │
│      int128       │
├───────────────────┤
│             17531 │
└───────────────────┘

In [6]:
df_com = duckdb.read_csv("./data/processed/communes_france_2025.csv")
df_com

┌────────────┬─────────────────────────┬─────────────────────────┬──────────┬──────────────────────┬──────────┬──────────────┬───────────┬───────────────────────────────────────────┬────────────┬────────────────┬─────────────────┐
│ code_insee │      nom_standard       │     nom_sans_accent     │ reg_code │       reg_nom        │ dep_code │   dep_nom    │ epci_code │                 epci_nom                  │ population │ superficie_km2 │ densite_hab_km2 │
│  varchar   │         varchar         │         varchar         │  int64   │       varchar        │ varchar  │   varchar    │  varchar  │                  varchar                  │   int64    │     int64      │     double      │
├────────────┼─────────────────────────┼─────────────────────────┼──────────┼──────────────────────┼──────────┼──────────────┼───────────┼───────────────────────────────────────────┼────────────┼────────────────┼─────────────────┤
│ 01001      │ L'Abergement-Clémenciat │ l-abergement-clemenciat │       84 

In [24]:
query = """ 
SELECT
    df_com.epci_code,
    SUM(df_mediation_num_grouped.nb_mediation) AS total_mediation_epci
FROM df_com
LEFT JOIN df_mediation_num_grouped
ON df_com.code_insee = df_mediation_num_grouped.code_insee
WHERE epci_code::VARCHAR NOT LIKE '%ZZZZ'
GROUP BY df_com.epci_code
"""

df_mediation_num_epci = duckdb.sql(query)
df_mediation_num_epci

┌───────────┬──────────────────────┐
│ epci_code │ total_mediation_epci │
│  varchar  │        int128        │
├───────────┼──────────────────────┤
│ 243400017 │                   66 │
│ 200070316 │                   12 │
│ 200096956 │                   29 │
│ 200093201 │                  382 │
│ 200084952 │                   87 │
│ 200067361 │                   23 │
│ 240300657 │                    2 │
│ 200068567 │                    8 │
│ 240700864 │                    7 │
│ 200068997 │                   31 │
│     ·     │                    · │
│     ·     │                    · │
│     ·     │                    · │
│ 246700959 │                    1 │
│ 246700926 │                    1 │
│ 246701098 │                    1 │
│ 200035103 │                 NULL │
│ 246800577 │                 NULL │
│ 241200674 │                 NULL │
│ 243301165 │                 NULL │
│ 200070571 │                 NULL │
│ 200034130 │                 NULL │
│ 243900610 │                 NULL │
├

In [9]:
df_epci = duckdb.read_csv("./data/processed/epci_france_2025.csv")
df_epci

┌───────────┬────────────────────────────────────────────────────────────┬────────────┬────────────────┬───────────┬────────────┬────────────────┬─────────────────┐
│ epci_code │                          epci_nom                          │ n_communes │ n_departements │ n_regions │ population │ superficie_km2 │ densite_hab_km2 │
│   int64   │                          varchar                           │   int64    │     int64      │   int64   │   int64    │     int64      │     double      │
├───────────┼────────────────────────────────────────────────────────────┼────────────┼────────────────┼───────────┼────────────┼────────────────┼─────────────────┤
│ 249740119 │ CA Intercommunale du Nord de la Réunion (CINOR)            │          3 │              1 │         1 │     213402 │            289 │          738.42 │
│ 249740101 │ CA Territoire de la Côte Ouest (TCO)                       │          5 │              1 │         1 │     215613 │            535 │          403.01 │
│ 24974009

In [27]:
duckdb.sql("SELECT count(*) FROM df_mediation_num_epci  ")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│         1255 │
└──────────────┘

In [25]:
query = """ 
SELECT 
    dmn.epci_code as siren,
    dmn.total_mediation_epci,
    round(10000 * dmn.total_mediation_epci / epci.population, 2) AS mediation_per_10k_habs 
FROM df_mediation_num_epci AS dmn
LEFT JOIN df_epci AS epci
ON dmn.epci_code = epci.epci_code
"""

df_mediation_num_final = duckdb.sql(query)
df_mediation_num_final

┌───────────┬──────────────────────┬────────────────────────┐
│   siren   │ total_mediation_epci │ mediation_per_10k_habs │
│  varchar  │        int128        │         double         │
├───────────┼──────────────────────┼────────────────────────┤
│ 249740119 │                   43 │                   2.01 │
│ 249740093 │                   16 │                   1.25 │
│ 249740085 │                   28 │                   2.11 │
│ 249740077 │                   58 │                    3.2 │
│ 249730052 │                    5 │                   5.84 │
│ 249730045 │                   30 │                   1.99 │
│ 249730037 │                   10 │                   1.02 │
│ 249720061 │                   14 │                   0.93 │
│ 249710070 │                   23 │                   3.02 │
│ 249710062 │                   12 │                   1.54 │
│     ·     │                    · │                     ·  │
│     ·     │                    · │                     ·  │
│     · 

In [26]:
df_mediation_num_final.write_csv("./data/processed/mediation_numerique.csv")